In [1]:
!pip install tensorflow-datasets

In [38]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [39]:
raw_train, raw_test = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[20%:]'],
    as_supervised = True
)

In [ ]:
FeaturesDict({
    'image':Image(shape=(224, 224, 3), dtype=tf.uint8),
    'image/filename':Text(shape=(), dtype=tf.string),
    'label':ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
})

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.image import ResizeMethod

In [ ]:
def preprocess(image, label):
    out_image = tf.image.resize(image, [224, 224], method=ResizeMethod.BICUBIC)
    out_image = tf.keras.applications.mobilenet_v2.preprocess_input(out_image) 
    return out_image, label

In [ ]:
batch_size = 32
train_batch = raw_train.map(preprocess).batch(batch_size)
test_batch = raw_test.map(preprocess).batch(batch_size)

In [ ]:
mobilenet_base = tf.keras.applications.MobileNetV2(  
    input_shape=(224, 224, 3),
    weights="imagenet",
    include_top=False)

In [ ]:
mobilenet_base.trainable=False

mobilenet_mobile = tf.keras.Sequential([
    mobilenet_base,
   tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1)
])

In [ ]:
mobilenet_mobile.compile(optimizer='adam',
                         loss = 'binary_crossentropy',
                         metrics = ['accuracy'])

In [48]:
mobilenet_mobile.fit(train_batch, epochs=5)

Epoch 1/5
582/582 [==============================] - 308s 529ms/step - loss: 0.2783 - accuracy: 0.9723
Epoch 2/5
582/582 [==============================] - 308s 529ms/step - loss: 0.1858 - accuracy: 0.9824
Epoch 3/5
582/582 [==============================] - 310s 532ms/step - loss: 0.1692 - accuracy: 0.9838
Epoch 4/5
582/582 [==============================] - 309s 531ms/step - loss: 0.1606 - accuracy: 0.9854
Epoch 5/5
582/582 [==============================] - 309s 531ms/step - loss: 0.1515 - accuracy: 0.9865


In [49]:
mobilenet_mobile.evaluate(test_batch, verbose=2)

582/582 - 309s - loss: 0.1809 - accuracy: 0.9821 - 309s/epoch - 532ms/step


[0.18089671432971954, 0.9820526838302612]